In [48]:

import imagesize
import bbox_utils
import numpy as np

def yolo_to_xyxy(img_path, yolo_pred):
    base_path = "/cluster/projects/vc/courses/TDT17/2022/open/RDD2022/Norway/test/images/"

    w, h = imagesize.get(base_path + img_path)
    
    yolo_pred = [float(e) for e in yolo_pred]
    
    bb = bbox_utils.BoundingBox.from_yolo(center=np.array([yolo_pred[1],yolo_pred[2]]),width=yolo_pred[3],height=yolo_pred[4], image_dimension=np.array([h,w]))
    class_idx = int(yolo_pred[0])
    if class_idx != 3:
        xyxy_pred = [class_idx + 1]
    else:
        xyxy_pred = [class_idx]
    
    bb = bb.to_xyxy()
    xyxy_pred.extend(bb[0])
    xyxy_pred.extend(bb[1])
    
    # if conf is included
    if len(yolo_pred) == 6:
        xyxy_pred.append(yolo_pred[5])
    
    return xyxy_pred
    

    
pred = ['1', '0.340242', '0.633268', '0.198463', '0.05242530']
yolo_to_xyxy("Norway_008161.jpg", pred)


[2, 880, 1241, 1604, 1348]

In [32]:
from operator import itemgetter

def sort_preds_by_conf(preds):
    sorted_preds = sorted(preds, key=itemgetter(5), reverse=True)#
    
    # # remove conf 
    # for pred in preds:
        
    #     pred.pop()
        
    return sorted_preds

In [47]:
from os import listdir, getcwd
from os.path import join
from tqdm import tqdm

test_files = "./data/RDD2022/Norway/images/test"
submission_folder = "./runs/detect/exp16/labels/"
# submission_file = open("./submission/submission_v3.txt", "w")

all_test_files = []
for f_name in listdir(test_files):
    all_test_files.append(f_name)

with open("./submission/submission_v6.txt", "w") as submission_file:
    for f_name in tqdm(listdir(submission_folder)):
        f = open(submission_folder + f_name, "r")
        img_name = f_name.split(".")[0] + ".jpg"

        preds = []
        for line in f:
            pred = line.replace("\n", "").split(" ")
            preds.append(yolo_to_xyxy(img_name, pred))
        
        preds = sort_preds_by_conf(preds)
        # threshold
        conf_threshold = 0.25
        
        filtered_preds = []
        for i, pred in enumerate(preds):
            if pred[-1] >= conf_threshold:
                pred.pop()
                filtered_preds.append(pred)
            
        filtered_preds = [str(item) for sublist in filtered_preds for item in sublist]
        
        
        submission_file.write(img_name + "," + " ".join(filtered_preds) + "\n")
        all_test_files.remove(img_name)
        
        

    for f_name in all_test_files:
        submission_file
        submission_file.write(f_name + ",\n")


100%|██████████| 971/971 [00:03<00:00, 302.59it/s]
